In [24]:
import numpy as np
import pandas as pd
import data_preprocessing as prep
import importlib

importlib.reload(prep)

<module 'data_preprocessing' from 'c:\\Users\\bmk1bj\\Documents\\GIT_repositories\\AIMatch\\data_preprocessing.py'>

### Ideas

- Split train and val randomly
- y = [home_goals - away_goals, total goals]
- model with two unrelated outputs
- sample_weight extended by tournament_group
- consider team strength trend this year (or this cup)


In [25]:
dataset = prep.Dataset()
X_train, Y_train, X_val, Y_val, X_test, sample_weights_train, sample_weights_val = dataset.get_input_data(label_weights=[1, 1], sample_weights_degree=2)

Number of relevant labeled matches: 18532/44060
X shape =  (18532, 457)
Y shape =  (18532, 2)
sample weights shape =  (18532,)
X shape =  (48, 457)
Y shape =  (48, 2)
sample weights shape =  (48,)


In [30]:
def score_sample(y_pred, y_true, label_weights):
    """ 
    Scoring according to the AI match rules
    y_pred = predictions
    y_true = ground truth 
    """
    
    y = np.around(y_pred / label_weights)
    y_gt = np.around(y_true / label_weights)
    #print(y, y_pred, y_true, y_gt)

    if y[0] == y_gt[0] and y[1] == y_gt[1]:
        return 4
    
    if y[0] == y_gt[0] and not y[0] == 0:
        return 3

    if (y[0] >= 0 and y_gt[0] >= 0) or (y[0] < 0 and y_gt[0] < 0):
        return 2   

    return 0

def score(Y, Y_gt, label_weights=[1, 1]):
    return np.sum([score_sample(Y[i], Y_gt[i], np.array(label_weights)) for i in range(len(Y))])

# Reference values
# = total score for validation data if results are hard-coded and all same without any prediction
# all models should overcome those values
max_score = 4 * len(Y_val)
ref_score_1 = score(np.zeros(Y_val.shape) * dataset.label_weights, Y_val, label_weights=dataset.label_weights) # 0:0
print("0:0", ref_score_1, "/", max_score, " - %s points per match" % (np.round(ref_score_1/len(Y_val), 2)))
ref_score_2 = score(np.ones(Y_val.shape) * dataset.label_weights, Y_val, label_weights=dataset.label_weights) # 1:0
print("1:0", ref_score_2, "/", max_score, " - %s points per match" % (np.round(ref_score_2/len(Y_val), 2)))
Y_pred = np.ones(Y_val.shape) * dataset.label_weights # 1:1
Y_pred[:, 0] = 0
ref_score_3 = score(Y_pred, Y_val, label_weights=dataset.label_weights) 
print("1:1", ref_score_3, "/", max_score, " - %s points per match" % (np.round(ref_score_3/len(Y_val), 2)))
Y_pred = np.ones(Y_val.shape) * dataset.label_weights # 0:1
Y_pred[:, 0] = -1
ref_score_3 = score(Y_pred, Y_val, label_weights=dataset.label_weights) 
print("0:1", ref_score_3, "/", max_score, " - %s points per match" % (np.round(ref_score_3/len(Y_val), 2)))
Y_pred = np.ones(Y_val.shape) # 2:1
Y_pred[:, 1] = 2
ref_score_3 = score(Y_pred * dataset.label_weights, Y_val, label_weights=dataset.label_weights) 
print("2:1", ref_score_3, "/", max_score, " - %s points per match" % (np.round(ref_score_3/len(Y_val), 2)))
Y_pred = 2 * np.ones(Y_val.shape) # 2:0
ref_score_3 = score(Y_pred * dataset.label_weights, Y_val, label_weights=dataset.label_weights) 
print("2:0", ref_score_3, "/", max_score, " - %s points per match" % (np.round(ref_score_3/len(Y_val), 2)))

0:0 8856 / 22240  - 1.59 points per match
1:0 9656 / 22240  - 1.74 points per match
1:1 9020 / 22240  - 1.62 points per match
0:1 4560 / 22240  - 0.82 points per match
2:1 9460 / 22240  - 1.7 points per match
2:0 9033 / 22240  - 1.62 points per match


In [31]:
def prediction_to_goals(y_weighted, label_weights):
    """ Encoding back to the exact result 
    [goal_diff, home_goals] -> [home_goals, away_goals]
    """
    y = y_weighted / np.array(label_weights)
    return np.hstack([y[1], y[1] - y[0]])

def predictions_to_goals(Y, label_weights):
    """ Encoding back to the exact result for whole data set
    [goal_diff, home_goals] -> [home_goals, away_goals]
    """
    return np.vstack([prediction_to_goals(Y[i], label_weights) for i in range(len(Y))])

In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import make_scorer



def show_predictions(X, Y, Y_pred, indexes, label_weights):
    for i in indexes:
        x = X[i]
        home_team_part = x[:len(dataset.all_teams)]
        away_team_part = x[len(dataset.all_teams):2*len(dataset.all_teams)]
        home_team = dataset.all_teams[np.where(home_team_part == 1)[0][0]]
        away_team = dataset.all_teams[np.where(away_team_part == 1)[0][0]]
        print(home_team, " x ", away_team, ": ", prediction_to_goals(Y_pred[i], label_weights), "-", prediction_to_goals(Y[i], label_weights), " ...................  output (weighted): ", Y_pred[i], Y[i], "   original: ", Y_pred[i]/label_weights, Y[i]/label_weights)


def fit_simple_regressor(X_train, Y_train, X_val, Y_val, X_test, sample_weights_train, regressor, name: str, label_weights: list, show_predicted_indexes = [0, 1], split_models=False):
    print(name)
    Y_val_pred, Y_test_pred = np.zeros(Y_val.shape), np.zeros((X_test.shape[0], Y_val.shape[1]))
    
    if split_models:
        try:
            regressor.fit(X_train, Y_train[:, 0].reshape(-1, 1),  sample_weight = sample_weights_train)        
        except:
            regressor.fit(X_train, Y_train[:, 0].reshape(-1, 1))
            print("Sample weights unused")
            
        Y_val_pred = regressor.predict(X_val).reshape(-1, 1)
        Y_test_pred = regressor.predict(X_test).reshape(-1, 1)
        
        try:
            regressor.fit(X_train, Y_train[:, 1].reshape(-1, 1),  sample_weight = sample_weights_train)        
        except:
            regressor.fit(X_train, Y_train[:, 1].reshape(-1, 1))
            print("Sample weights unused")
        Y_val_pred = np.hstack((Y_val_pred, regressor.predict(X_val).reshape(-1, 1)))
        Y_test_pred = np.hstack((Y_test_pred, regressor.predict(X_test).reshape(-1, 1)))
        
    else:
        try:
            regressor.fit(X_train, Y_train,  sample_weight = sample_weights_train)        
        except:
            regressor.fit(X_train, Y_train)
            print("Sample weights unused")
        
        Y_val_pred = regressor.predict(X_val)
        Y_test_pred = regressor.predict(X_test)
        
        
    reg_score = score(np.round(Y_val_pred), Y_val, label_weights=label_weights)
    print(name)
    print(reg_score, "/", max_score, " - %s points per match" % (np.round(reg_score/len(Y_val), 2)))
    
    print("Val:")
    show_predictions(X_val, Y_val, Y_val_pred, show_predicted_indexes, label_weights)
    print("Test:")
    show_predictions(X_test, np.zeros(Y_test_pred.shape), Y_test_pred, show_predicted_indexes, label_weights)
    

models = {
    #"Tree": DecisionTreeRegressor(random_state=0),
    "Linear": LinearRegression(),
    "MLP": MLPRegressor(hidden_layer_sizes = (128, 32, 8), activation="identity")
}


trainable = 0.1
start = int((1 - trainable) * len(X_train))
for name, model in models. items():
    fit_simple_regressor(X_train[start:], Y_train[start:], X_val, Y_val, X_test,
                         sample_weights_train[start:], model, name, 
                         show_predicted_indexes = [r for r in range(12,24)], label_weights=dataset.label_weights,
                         split_models = True
                         )


Linear
Linear
9470 / 22240  - 1.7 points per match
Val:
Nigeria  x  Ghana :  [1.77148438 1.84570312] - [1. 0.]  ...................  output (weighted):  [-0.07421875  1.77148438] [1. 1.]    original:  [-0.07421875  1.77148438] [1. 1.]
Zimbabwe  x  Senegal :  [3.58707674e+11 2.75740670e+10] - [2. 4.]  ...................  output (weighted):  [3.31133607e+11 3.58707674e+11] [-2.  2.]    original:  [3.31133607e+11 3.58707674e+11] [-2.  2.]
Egypt  x  Morocco :  [2.41992188 1.15429688] - [0. 0.]  ...................  output (weighted):  [1.265625   2.41992188] [0. 0.]    original:  [1.265625   2.41992188] [0. 0.]
Cameroon  x  Togo :  [1.05078125 1.1484375 ] - [2. 0.]  ...................  output (weighted):  [-0.09765625  1.05078125] [2. 2.]    original:  [-0.09765625  1.05078125] [2. 2.]
Ecuador  x  Honduras :  [2.5      1.390625] - [1. 0.]  ...................  output (weighted):  [1.109375 2.5     ] [1. 1.]    original:  [1.109375 2.5     ] [1. 1.]
Finland  x  South Korea :  [1.95117188 

c:\Users\bmk1bj\.conda\envs\aimatch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Sample weights unused


c:\Users\bmk1bj\.conda\envs\aimatch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Sample weights unused
MLP
9568 / 22240  - 1.72 points per match
Val:
Nigeria  x  Ghana :  [1.73392692 1.59527168] - [1. 0.]  ...................  output (weighted):  [0.13865525 1.73392692] [1. 1.]    original:  [0.13865525 1.73392692] [1. 1.]
Zimbabwe  x  Senegal :  [1.44860759 2.21675082] - [2. 4.]  ...................  output (weighted):  [-0.76814323  1.44860759] [-2.  2.]    original:  [-0.76814323  1.44860759] [-2.  2.]
Egypt  x  Morocco :  [2.31517371 0.93529114] - [0. 0.]  ...................  output (weighted):  [1.37988257 2.31517371] [0. 0.]    original:  [1.37988257 2.31517371] [0. 0.]
Cameroon  x  Togo :  [1.03478286 0.99596118] - [2. 0.]  ...................  output (weighted):  [0.03882169 1.03478286] [2. 2.]    original:  [0.03882169 1.03478286] [2. 2.]
Ecuador  x  Honduras :  [2.53717031 1.40304999] - [1. 0.]  ...................  output (weighted):  [1.13412032 2.53717031] [1. 1.]    original:  [1.13412032 2.53717031] [1. 1.]
Finland  x  South Korea :  [2.04282495 2.7